In [2]:
import os
import time
import copy
import json
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import Subset

from torch.utils.data import ConcatDataset #unseen data를 통해서 언러닝 재학습에서 사용.
from torch.utils.data import TensorDataset

from options import args_parser
from update import LocalUpdate, test_inference
from models import CNNMnist, Generator, Discriminator, generate_images, filter_images
from utils import get_dataset, average_weights, exp_details, create_poisoned_dataset
from unlearn import (
    train_generator_ungan,
    SyntheticImageDataset,
    partition_synthetic_data_iid,
    get_synthetic_subset
)
from evaluate_mia import evaluate_mia

In [4]:
def move_dataset_to_device(dataset, device):
    images = []
    labels = []
    for x, y in dataset:
        images.append(x.to(device))
        labels.append(torch.tensor(y).to(device))
    return TensorDataset(torch.stack(images), torch.stack(labels))


def add_backdoor_trigger(x):
    x_bd = x.clone()
    x_bd[:, 25:28, 25:28] = 0.9
    return x_bd

def evaluate_backdoor_asr(model, dataset, target_label, device):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for i in range(len(dataset)):
            x, y = dataset[i]
            # 백도어 트리거 삽입
            x_bd = add_backdoor_trigger(x).to(device)
            x_bd = x_bd.unsqueeze(0)  # 배치 차원 추가

            output = model(x_bd)
            pred = output.argmax(dim=1).item()

            total += 1
            if pred == target_label:
                correct += 1

    asr = correct / total
    return asr

def select_model(args, train_dataset):
    if args.model == 'cnn':
        return CNNMnist(args=args)
    else:
        raise NotImplementedError


In [5]:
def main():
    start_time = time.time()
    args = args_parser()
    device = 'cuda' if args.gpu and torch.cuda.is_available() else 'cpu'

    exp_details(args)

    # ===================== 1. 데이터셋 로딩 및 초기화 =====================
    train_dataset, test_dataset,unseen_dataset, user_groups = get_dataset(args)
    

    full_dataset, user_groups = create_poisoned_dataset(train_dataset, user_groups, args,
                                                        malicious_client=0,
                                                        target_label=6,
                                                        poison_ratio=0.8)

    global_model = select_model(args, full_dataset).to(device)
    global_model.train()

    generator = Generator(z_dim=args.z_dim).to(device)
    discriminator = Discriminator().to(device)

    global_weights = global_model.state_dict()
    train_loss, train_accuracy = [], []

    forget_client = 0
    forget_idxs = user_groups[forget_client]
    retain_idxs = [i for i in range(len(train_dataset)) if i not in forget_idxs]
    test_idxs = np.random.choice(len(test_dataset), len(forget_idxs), replace=False)

    # ===================== 2. 연합 학습 (FedAvg) =====================
    for epoch in tqdm(range(args.epochs), desc='Global Training Rounds'):
        print(f'\n| Global Training Round : {epoch + 1} |')

        local_weights, local_losses = [], []
        m = max(int(args.frac * args.num_users), 1)
        idxs_users = np.random.choice(range(args.num_users), m, replace=False)

        for idx in idxs_users:
            if idx == forget_client:
                continue  # 언러닝 요청자 제거 / 재학습

            local_model = LocalUpdate(args=args, dataset=full_dataset, idxs=user_groups[idx])
            w, loss = local_model.update_weights(model=copy.deepcopy(global_model), global_round=epoch)
            local_weights.append(copy.deepcopy(w))
            local_losses.append(loss)

        global_weights = average_weights(local_weights)
        global_model.load_state_dict(global_weights)

        loss_avg = sum(local_losses) / len(local_losses)
        acc, _ = test_inference(args, global_model, test_dataset)
        train_loss.append(loss_avg)
        train_accuracy.append(acc)

        print(f"Training Loss: {loss_avg:.4f} | Train Accuracy: {acc*100:.2f}%")
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Re_Training Time: {elapsed_time:.2f}초")
    test_acc_before, test_loss_before = test_inference(args, global_model, test_dataset)
    print(f"\n[Test Before Unlearning] Accuracy: {test_acc_before*100:.2f}% | Loss: {test_loss_before:.4f}")
    print("[MIA] Evaluating membership inference attack...")
    
    all_idxs = set(range(len(full_dataset)))
    non_member_candidates = list(all_idxs - set(forget_idxs))
    #여기에서 쉐도우에는 forget 데이터가 없도록 하기.
    mia_result = evaluate_mia(
        model=global_model,
        dataset=full_dataset,
        test_dataset= test_dataset,
        forget_idxs=forget_idxs,
        retain_idxs=test_idxs,
        shadow_idxs=np.random.choice(non_member_candidates, len(forget_idxs), replace=False),
        device=device,
        save_path="./mia_result_before.json"
    )

    print(f"[MIA] AUC: {mia_result['auc']:.4f}")
    print("\n[Backdoor Attack Success Rate Evaluation]")
    target_label = 6  # 공격 대상 라벨 (main() 함수와 맞춰야 함)
    asr = evaluate_backdoor_asr(global_model, test_dataset, target_label, device)
    print(f"Backdoor Attack Success Rate (ASR): {asr*100:.2f}%")

    #torch.save(global_model.state_dict(), args.save_model)
    #print(f"[Saved] model to {args.save_model}\n")

In [ ]:
import sys

sys.argv = [
    'ipykernel_launcher.py',
    '--epochs', '100',
    '--num_users', '10',
    '--frac', '1.0',
    '--local_ep', '10',
    '--local_bs', '64',
    '--lr', '0.01',
    '--momentum', '0.75',
    '--dataset', 'mnist',
    '--model', 'cnn',
    '--iid', '1',
    '--gpu', '0',
    '--num_classes', '10',
    '--load_model', 'None',
    '--save_model', './saved_models/model.pth',
    '--z_dim', '100',
    '--gen_threshold', '0.7',
    '--num_gen_samples', '128'
]


In [7]:
main()

===== Experiment Settings =====
Model           : cnn
Dataset         : mnist
Num Clients     : 10
Fraction        : 1.0
IID             : 1
Local Epochs    : 10
Batch Size      : 64
Learning Rate   : 0.01
Generator z_dim : 100
Disc. Threshold : 0.7


Global Training Rounds:   0%|          | 0/200 [00:00<?, ?it/s]


| Global Training Round : 1 |


Global Training Rounds:   0%|          | 1/200 [00:10<33:27, 10.09s/it]

Training Loss: 0.2116 | Train Accuracy: 98.03%

| Global Training Round : 2 |


...

Global Training Rounds: 100%|█████████▉| 199/200 [31:02<00:09,  9.22s/it]

Training Loss: 0.0000 | Train Accuracy: 99.09%

| Global Training Round : 200 |


Global Training Rounds: 100%|██████████| 200/200 [31:11<00:00,  9.36s/it]

Training Loss: 0.0000 | Train Accuracy: 99.08%
Re_Training Time: 1876.33초



[Test Before Unlearning] Accuracy: 99.08% | Loss: 0.0438
[MIA] Evaluating membership inference attack...
retain confidence mean: 0.99997807
forget confidence mean: 0.9983963
evalu retain confidence mean: 0.9978992
evalu forget confidence mean: 0.9983963
[MIA] AUC: 0.5014

[Backdoor Attack Success Rate Evaluation]
Backdoor Attack Success Rate (ASR): 9.50%
